# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Process the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the 
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
!ls

event_data  event_datafile_new.csv  images  Project_1B_ Project_Template.ipynb


# Part II. Apache Cassandra 

#### The <font color=red>event_datafile_new.csv</font> contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> <br>

<img src="images/image_event_datafile_new.jpg">

#### Create a Cluster

In [6]:
# Establish a connection to a Cassandra instance on the local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

- Obtain the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


- Obtain the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

- Obtain every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1

In [9]:
# sneak peak of the header
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    for line in csvreader:
        print(line)
        break

['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']


In [10]:
# Query 1: Obtain the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

# session_id and item_in_session are used as part of the primary key since we need to filter by those two columns
query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(session_id INT, item_in_session INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [12]:
# session_id and item_in_session are used as part of the primary key since we need to filter by those two columns
query = """
    SELECT
        artist, song, length
    FROM
        song_library
    WHERE
        session_id = 338 AND item_in_session = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist: ", row.artist, "\nSong: ", row.song, "\nLength: ", row.length)

Artist:  Faithless 
Song:  Music Matters (Mark Knight Dub) 
Length:  495.30731201171875


#### Query 2

In [13]:
# Query 2: Obtain name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Since we are filtering by two columns, a composite partition key (user_id, session_id) is used to improve performance
# the composite partition key breaks the data into smaller chuncks 
# item_in_session is used as clustering column to ensure Primary Key uniqueness
query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, first_name TEXT, last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)


# Insertion
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_library (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

        
# SELECT statement to verify the data was entered into the table
# Since we are filtering by two columns, a composite partition key (user_id, session_id) is used to improve performance
# the composite partition key breaks the data into smaller chuncks 
# item_in_session is used as clustering column to ensure Primary Key uniqueness
query = """
    SELECT
        artist, 
        song, 
        first_name,
        last_name
    FROM
        artist_library
    WHERE
        user_id = 10 AND session_id = 182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist: ", row.artist, "\nSong: ", row.song, "\nUser First Name: ", row.first_name, "\nUser Last Name: ", row.last_name)

Artist:  Down To The Bone 
Song:  Keep On Keepin' On 
User First Name:  Sylvie 
User Last Name:  Cruz
Artist:  Three Drives 
Song:  Greece 2000 
User First Name:  Sylvie 
User Last Name:  Cruz
Artist:  Sebastien Tellier 
Song:  Kilometer 
User First Name:  Sylvie 
User Last Name:  Cruz
Artist:  Lonnie Gordon 
Song:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
User First Name:  Sylvie 
User Last Name:  Cruz


#### Query 3

In [15]:
# Query 3: Obtain every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

# song is used as partition key since we need to filter by that column
# user_id is used as clustering column to ensure uniqueness of the primary key
query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + "(song TEXT, user_id INT, first_name TEXT, last_name TEXT, PRIMARY KEY ((song), user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

# Insertion
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_library (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
# SELECT statement to verify the data was entered into the table
# song is used as partition key since we need to filter by that column
# user_id is used as clustering column to ensure uniqueness of the primary key
query = """
    SELECT
        first_name, 
        last_name
    FROM
        user_library
    WHERE
        song = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("User first name: ", row.first_name, "\nUser last name: ", row.last_name)            

User first name:  Jacqueline 
User last name:  Lynch
User first name:  Tegan 
User last name:  Levine
User first name:  Sara 
User last name:  Johnson


### Drop the tables before closing out the sessions

In [16]:
session.execute("DROP TABLE IF EXISTS song_library")
session.execute("DROP TABLE IF EXISTS artist_library")
session.execute("DROP TABLE IF EXISTS user_library")

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()